In [1]:
import numpy as np
import time
import scipy.stats as stats
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
import matplotlib.pyplot as plt
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)

In [2]:
data = pd.read_sas('/home/guangya/Downloads/wnv_2245new.sas7bdat') # In the Cook_Dupage Directory

In [8]:
data.head()

,hexid,Input_FID,tempc,preci,yr,weeks,yrweeks,yrwksfid,templag1,templag2,...,wwpct,ehwpct,yr_hexid,Jantemp,PopYesNo,hpctpreww,hpctpostww,hpct7089,hpctpost90,income1
0,1.0,130.0,16.9758,6.046130,2005.0,22.0,b'200522',b'200522130',15.9957,13.7796,...,27.184466,0.0,"b'2005,1'",-3.815482,1.0,0.0,0.0,0.0,0.0,41.979816
1,1.0,130.0,25.3449,58.968242,2005.0,23.0,b'200523',b'200523130',16.9758,15.9957,...,27.184466,0.0,"b'2005,1'",-3.815482,1.0,0.0,0.0,0.0,0.0,41.979816
2,1.0,130.0,21.0872,15.114200,2005.0,24.0,b'200524',b'200524130',25.3449,16.9758,...,27.184466,0.0,"b'2005,1'",-3.815482,1.0,0.0,0.0,0.0,0.0,41.979816
3,1.0,130.0,22.3485,0.000000,2005.0,25.0,b'200525',b'200525130',21.0872,25.3449,...,27.184466,0.0,"b'2005,1'",-3.815482,1.0,0.0,0.0,0.0,0.0,41.979816
4,1.0,130.0,25.7740,6.371029,2005.0,26.0,b'200526',b'200526130',22.3485,21.0872,...,27.184466,0.0,"b'2005,1'",-3.815482,1.0,0.0,0.0,0.0,0.0,41.979816


In [3]:
x_test = data[data.drop(columns=['wnvbinary','yrweeks','yrwksfid','yr_hexid','year']).columns[:]]

In [7]:
x_test = x_test[(x_test.weeks >= 22) & (x_test.weeks <= 31)]

,hexid,Input_FID,tempc,preci,yr,weeks,templag1,templag2,templag3,templag4,...,ccpct,wwpct,ehwpct,Jantemp,PopYesNo,hpctpreww,hpctpostww,hpct7089,hpctpost90,income1
0,1.0,130.0,16.975800,6.046130,2005.0,22.0,15.995700,13.779600,16.730600,9.073000,...,0.416089,27.184466,0.0,-3.815482,1.0,0.0,0.000000,0.000000,0.000000,41.979816
1,1.0,130.0,25.344900,58.968242,2005.0,23.0,16.975800,15.995700,13.779600,16.730600,...,0.416089,27.184466,0.0,-3.815482,1.0,0.0,0.000000,0.000000,0.000000,41.979816
2,1.0,130.0,21.087200,15.114200,2005.0,24.0,25.344900,16.975800,15.995700,13.779600,...,0.416089,27.184466,0.0,-3.815482,1.0,0.0,0.000000,0.000000,0.000000,41.979816
3,1.0,130.0,22.348500,0.000000,2005.0,25.0,21.087200,25.344900,16.975800,15.995700,...,0.416089,27.184466,0.0,-3.815482,1.0,0.0,0.000000,0.000000,0.000000,41.979816
4,1.0,130.0,25.774000,6.371029,2005.0,26.0,22.348500,21.087200,25.344900,16.975800,...,0.416089,27.184466,0.0,-3.815482,1.0,0.0,0.000000,0.000000,0.000000,41.979816
5,1.0,130.0,22.277000,3.417303,2005.0,27.0,25.774000,22.348500,21.087200,25.344900,...,0.416089,27.184466,0.0,-3.815482,1.0,0.0,0.000000,0.000000,0.000000,41.979816
6,1.0,130.0,24.915200,10.102133,2005.0,28.0,22.277000,25.774000,22.348500,21.087200,...,0.416089,27.184466,0.0,-3.815482,1.0,0.0,0.000000,0.000000,0.000000,41.979816
7,1.0,130.0,25.904900,18.433824,2005.0,29.0,24.915200,22.277000,25.774000,22.348500,...,0.416089,27.184466,0.0,-3.815482,1.0,0.0,0.000000,0.000000,0.000000,41.979816
8,1.0,130.0,23.836800,45.339948,2005.0,30.0,25.904900,24.915200,22.277000,25.774000,...,0.416089,27.184466,0.0,-3.815482,1.0,0.0,0.000000,0.000000,0.000000,41.979816
9,1.0,130.0,24.750600,5.782040,2005.0,31.0,23.836800,25.904900,24.915200,22.277000,...,0.416089,27.184466,0.0,-3.815482,1.0,0.0,0.000000,0.000000,0.000000,41.979816


In [4]:
x_selected = data[data.drop(columns=['wnvbinary','yrweeks','yrwksfid','yr_hexid','year']).columns[:]].values
y_selected = data['wnvbinary'].values

In [5]:
trainX_sel, testX_sel, trainY_sel, testY_sel = train_test_split(x_selected, y_selected, test_size = 0.2, shuffle = True) # CV

In [ ]:
model_RF_best_2 = RandomForestRegressor(n_estimators=1500,
                                 n_jobs = -1,
                                 max_features=None,
                                 max_depth=None,
                                 bootstrap=True,
                                 min_samples_leaf=2
                                 )
model_RF_best_2.fit(trainX_sel, trainY_sel)

In [ ]:
time_start = time.time()
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, None, 110],
    'max_features': ['sqrt', 4],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [500, 2000, 4000, 1000]
}
CV_model_RF_3 = GridSearchCV(model_RF_best_2, param_grid, cv=5)
CV_model_RF_3.fit(trainX_sel, trainX_sel)
print("time consumed:", time.time() - time_start)

In [2]:
def model_RF_test(model_RF, dataX, dataY):
    print("Model performance")
    predict_data = model_RF.predict_proba(dataX)
    
    # Some stats
    print("Feature Importantce : ")
    print(model_RF.feature_importances_)
    print("Total number of WNV occurence in test set : " + str(len(dataY[dataY > 0])))
    
    print("Number of WNV occurence the model is able to capture in test set:" + str(dataY[np.where(predict_data[:,1]  > 0)].sum()))
    
    print("Log loss : " + str(log_loss(dataY,predict_data)))
    
    print("This is to test the performance of random forest model, ideally, the logloss is low and also it is able to capture most of the WNV occurence")
    
    return None # Check how many wnv it predicts

In [ ]:
model_RF_test(CV_model_RF_3,testX_sel,testY_sel)